# Pregunta 1

In [42]:
# Auxiliary functions

# Convert from plain text to corresponding positions in the alphabet
def text_to_numbers(text, characters):
    return [characters.index(c) for c in text]

# Convert from alphabet positions to the corresponding plain text
def numbers_to_text(numbers, characters):
    return ''.join([characters[n] for n in numbers])

# Encrypt plain text using RP
def encrypt(text, key, alphabet):
    numbered_text = text_to_numbers(text, alphabet)
    numbered_key = text_to_numbers(key, alphabet)
    encrypted = []
    for idx, n in enumerate(numbered_text):
        encrypted.append((n + numbered_key[idx % len(key)]) % len(alphabet))
    return numbers_to_text(encrypted, alphabet)

# Calculate absolute distance between a string and the distribution of letters over an alphabet
def abs_distance(string, frequencies):
    return sum([abs(frequencies[c] - string.count(c) / len(string)) for c in frequencies])

In [48]:
# Break Repeated Pad using frequency analysis
def break_rp(ciphertext, frequencies, distance):
    """
    Arguments:
        ciphertext: An arbitrary string representing the encrypted version of a plaintext.
        frequencies: A dictionary representing a character frequency over the alphabet.
        distance: A function indicating how distant is a string from following a character frequency.
    Returns:
        key: A guess of the key used to encrypt the ciphertext, assuming that the plaintext message was written in a language in which
        letters distribute according to frequencies.
    """
    print(ciphertext)
    return 'hola1234'

In [49]:
# TODO: more examples and better testing with files
# Test Repeated Pad breaking
if __name__ == '__main__':
    # Letter frequency for the english alphabet
    english_freq = {'a': 8.17,
                    'b': 1.29,
                    'c': 2.76,
                    'd': 4.25,
                    'e': 12.88,
                    'f': 2.23,
                    'g': 2.02,
                    'h': 6.09,
                    'i': 6.97,
                    'j': 0.15,
                    'k': 0.77,
                    'l': 4.03,
                    'm': 2.41,
                    'n': 6.75,
                    'o': 7.51,
                    'p': 1.93,
                    'q': 0.1,
                    'r': 5.99,
                    's': 6.33,
                    't': 9.06,
                    'u': 2.78,
                    'v': 0.98,
                    'w': 2.36,
                    'x': 0.15,
                    'y': 1.97,
                    'z': 0.07}

    # Plain text of 1065 characters
    plain_text = 'fifteenhoursintoeldenringidefeatedgodrickthefirstoffiveeldenlordsinthetimebetweenemergingintothelandsbetweenandstrikinghimdownihaddiscovereddecrepitruinsventuredintotwistingcavesstumbleduponenemyencampmentsandbattledtoothandnailagainstchallengingbossesfromsoftwaresgameshavealwaysmadeyoufeelsmallinmanywaystheytellyouthatyouareworthlessaplagueriddenratoraccursedundeadunfiteventobecinderstheyaskyoutonavigateunflinchingbrutalworldsandpityouagainstenemiesthatsystematicallydismantleyouregoeldenringmaintainsthenailbitingcombatandairofmysterythathasdistinguishedfromsoftwaressoulsbornegamesbutitiselevatedtonewheightsbythestudiosinterpretationofwhatanopenworldgamecanbehavingbroughtdowngodrickthebreadthoftheworldandthewayinwhichfromsoftwarehasapplieditssignaturestyletoanopenworldwasonfulldisplayreinforcinghowinsignificantireallywasanddrivinghomethemagnitudeofthetaskthatstillawaitedme'

    # Random key of 21 characters
    rp_key = 'fjsncitnwiofksotksgfb'

    # Encrypted text
    cipher_text = encrypt(plain_text, rp_key, list(english_freq))

    # Break RP test
    break_rp(cipher_text, english_freq, abs_distance)

krxggmgukcfxsfhhodjjowrftklxsaihjnycwbaipumnxvtambbnwaowzwofrtsibaavpxgeusgolkxofkrfwpaaiqggkbvjvsbwctkyxjnfnpllgnqynxyvbwvubonqsqfqlpkdswovrxmjkujyamvpaorjbiwovwgdgzbjxcaaiktiaagyeepeovaupsnfrogxayiauwwbmcsticfclyglmbkbvfxvbtsdglbnwkgeptyhmblsfuuykyjtkagzuwygsifjcyofoknfwjjdjcglzwlsdymtxodyrbquaaoiglsimxdzsrdwrqztdlucbrbqifjggfmrdkxtfydnicxeelrjxjomyjghdzakrfcgqairzxxwmonksutkwpkvwrnahmoqoluquzutwsikotgacbkvabvratlcwdlnnehehlgfxvdbdquzbljaaubxaauwjclvtdkexujvsgkktyhgrnceogddkdpzawtqmeqavfnxyatsfzfjsblugvtvhjwysfuvyehfufwvnkzhsiggyojmmrszmbxmafvqgtqqgmovtkyeytgyfsegalbqtggyjbxqssjtgdlvvqlrhmjfdwrmyfkbijryuvaulppsxdmrbykosujahegbtgewbtpovtdsttqjwobttwtwushkfpxrsbnolkjbwoagzwksqgrksuqyijkjrclmuknhmoockvvgseyqwjcgbaspwhrxfhwkukubjjrjilnlxznovwmckolofcmegamlhmhtkfciofctsqmonuwgsqtziskdekqxjjsogeeqgtdwknxkwzxalndfwlvtmtyhgkfcsbwnjoajspzbommuauolxahnnwukumnlnusmuwbgysdztgsoyfivw
